In [1]:
import requests as req
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import psycopg2
import datetime as dt
from tabulate import tabulate
pd.set_option("display.max_columns", None)

### INDICE DE CANASTA BÁSICA A PARTIR DE LOS PRECIOS DE LOS PRODUCTOS EN COTO DIGITAL.

El listado de productos fue obtenido de la pagina del INDEC https://www.indec.gob.ar/ftp/cuadros/sociedad/EPH_metodologia_22_pobreza.pdf y los valores se obtienen de la pagina de Coto Digital https://www.cotodigital3.com.ar/ 
Se utiliza coto digital porque es la cadena de supermercados que tiene mayor cobertura en el país. Podriamos utilizar cualquier otra cadena de supermercados que cuenten con precios publicados en internet.

La idea es tener una referencia de la variacion de los precios de la canasta basica para poder constatarlos contra los datos del INDEC que se publican mensualmente. 
Es una recoleccion muy sencilla de datos sin tener en cuenta ubicacion geoestadistica, por lo que no es una muestra representativa de la poblacion.

Funciones para obtener los productos de la canasta basica de Coto Digital y almacenarlos en un archivo en un diccionario.

In [2]:
listado = {'fecha': dt.datetime.now().strftime("%Y-%m-%d")}

def kilo(nombre_producto, producto_url, porcion = 1):
    valor = BeautifulSoup(producto_url.text, 'html.parser')
    valor = valor.find_all('span', class_='unit')

    
    nombre = nombre_producto.replace(" ", "_")
    
    try:
        valor = valor[0].get_text()
    except IndexError:
        listado.update({nombre: 0})
        print(f'{nombre} IndexError, {listado[nombre]}')
        return None
    
    match = re.search(r"\$([\d,.]+)", valor)
        
    if match:
        number = float(match.group(1).replace(".", "").replace(",", "."))
        print(nombre, (number * porcion)) 
        
        listado.update({nombre: (number * porcion)})    
    else:
        listado.update({nombre: 0})
        print("No se encontró un número en el string")
    
        
    

def unidad(nombre_producto, producto_url):
    valor = BeautifulSoup(producto_url.text, 'html.parser')
    valor = valor.find_all('span', class_='atg_store_newPrice')
    
    nombre = nombre_producto.replace(" ", "_")

    try:
        valor = valor[0].get_text()
    except IndexError:
        listado.update({nombre: 0})
        print(f'{nombre} IndexError, {listado[nombre]}')
        return None
    
    match = re.search(r"\$([\d,.]+)", valor)
        
    if match:
        number = float(match.group(1).replace(".", "").replace(",", "."))
        print(nombre, number)
        listado.update({nombre: number})
            
    else:
        listado.update({nombre: 0})
        print("No se encontró un número en el string")
    





Carga CSV con enlaces a los productos de la canasta basica de Coto Digital

In [4]:
canasta_basica = pd.read_csv('listado canasta basica.csv', sep=';', encoding='latin-1', usecols=['producto', 'cantidad_g_ml', 'url_coto', 'tipo_producto', 'porcion'])
canasta_basica.head(10)

,producto,cantidad_g_ml,url_coto,tipo_producto,porcion
0,Pan Frances,"6,75",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
1,Galletitas dulces,210,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,NaN
2,Galletitas de agua,420,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,NaN
3,Harina de trigo,"1,08",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
4,Harina de maiz,210,https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,0.25
5,Arroz,"1,2",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,1.20
6,Fideos secos,"1,74",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
7,Asado,"1,23",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,1.20
8,Carnaza comun,510,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,0.50
9,Hueso con carne,390,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,0.39


Leemos cada enlace y mediante las funciones de web scraping obtenemos los precios de los productos y lo guardamos en un diccionario.

In [5]:

for producto in canasta_basica.index:
  
   if canasta_basica.loc[producto, 'tipo_producto'] == 'kilo':
      url = req.get(canasta_basica.loc[producto, 'url_coto'])
      
      if url.status_code == 200:
         kilo(canasta_basica.loc[producto, 'producto'], url)
      else:
         print('no hay url')
      
   elif canasta_basica.loc[producto, 'tipo_producto'] == 'unidad':
      url = req.get(canasta_basica.loc[producto, 'url_coto'])
      
      if url.status_code == 200:
         unidad(canasta_basica.loc[producto, 'producto'], url)
      else:
         print('no hay url')
      
   else:
      canasta_basica.loc[producto, 'url_coto'] == 'nan'
      print(producto, 'no hay url')

   


Pan_Frances 1284.71
Galletitas_dulces 221.91
Galletitas_de_agua 379.25
Harina_de_trigo IndexError, 0
Harina_de_maiz 264.55
Arroz 334.92
Fideos_secos IndexError, 0
Asado 1699.9
Carnaza_comun 1199.9
Hueso_con_carne 599.9
Paleta 1589.9
Carne_picada 1499.9
Nalga 1889.9
Higado 379.9
Pechito_de_cerdo 1189.9
Pollo 659.9
Carne_de_pescado 2299.0
Mortadela 1489.0
Paleta_cocida 2445.0
Salchich¢n 1325.0
Salame 4533.33
Aceite_de_girasol 623.13
Margarina_para_cocinar 317.96
Leche_fluida 278.22
Leche_en_polvo_entera 1610.64
Queso_crema 272.65
Queso_cuartirolo 1599.0
Queso_de_rallar 3999.0
Manteca IndexError, 0
Yogur IndexError, 0
Dulce_de_leche 997.5
Huevo 2080.0
Manzana 419.0
33 no hay url
Naranja 469.0
Banana 299.0
Pera 199.0
Batata 849.0
Papa 279.0
Acelga 199.0
Cebolla 199.0
Choclo 499.0
Lechuga 699.0
Tomate_perita 1219.0
Zanahoria 299.0
Zapallo 149.0
Tomate_envasado IndexError, 0
Arvejas_en_lata 642.85
Lentejas_secas 1894.95
Azucar 689.64
Dulce_de_batata 753.58
Mermelada 748.83
Sal_fina 363.5
May

In [6]:
listado

{'fecha': '2023-03-27',
 'Pan_Frances': 1071.42,
 'Galletitas_dulces': 198.49,
 'Galletitas_de_agua': 349.99,
 'Harina_de_trigo': 201.75,
 'Harina_de_maiz': 229.02,
 'Arroz': 223.89,
 'Fideos_secos': 355.5,
 'Asado': 1599.9,
 'Carnaza_comun': 1049.9,
 'Hueso_con_carne': 529.9,
 'Paleta': 1499.9,
 'Carne_picada': 1349.9,
 'Nalga': 1749.9,
 'Higado': 299.9,
 'Pechito_de_cerdo': 1049.9,
 'Pollo': 479.9,
 'Carne_de_pescado': 1690.0,
 'Mortadela': 1630.0,
 'Paleta_cocida': 1909.0,
 'Salchich¢n': 1325.0,
 'Salame': 4263.33,
 'Aceite_de_girasol': 574.74,
 'Margarina_para_cocinar': 277.05,
 'Leche_fluida': 242.15,
 'Leche_en_polvo_entera': 1485.55,
 'Queso_crema': 264.2,
 'Queso_cuartirolo': 1469.0,
 'Queso_de_rallar': 2699.0,
 'Manteca': 1592.0,
 'Yogur': 236.25,
 'Dulce_de_leche': 736.12,
 'Huevo': 1361.0,
 'Manzana': 449.0,
 'Naranja': 599.0,
 'Banana': 449.0,
 'Pera': 189.0,
 'Batata': 499.0,
 'Papa': 269.0,
 'Acelga': 329.0,
 'Cebolla': 199.0,
 'Choclo': 199.0,
 'Lechuga': 1199.0,
 'Tomat

### Cargar todo a la base de datos postgres de manera local creada en un contenedor docker

Creo el diccionario de productos para la creacion de la tabla. 

In [7]:
tabla = {'id' : 'SERIAL PRIMARY KEY', 'fecha' : 'DATE' }

for producto in listado:
    if producto == 'fecha':
        continue
    tabla[producto] = 'FLOAT'

tabla

{'id': 'SERIAL PRIMARY KEY',
 'fecha': 'DATE',
 'Pan_Frances': 'FLOAT',
 'Galletitas_dulces': 'FLOAT',
 'Galletitas_de_agua': 'FLOAT',
 'Harina_de_trigo': 'FLOAT',
 'Harina_de_maiz': 'FLOAT',
 'Arroz': 'FLOAT',
 'Fideos_secos': 'FLOAT',
 'Asado': 'FLOAT',
 'Carnaza_comun': 'FLOAT',
 'Hueso_con_carne': 'FLOAT',
 'Paleta': 'FLOAT',
 'Carne_picada': 'FLOAT',
 'Nalga': 'FLOAT',
 'Higado': 'FLOAT',
 'Pechito_de_cerdo': 'FLOAT',
 'Pollo': 'FLOAT',
 'Carne_de_pescado': 'FLOAT',
 'Mortadela': 'FLOAT',
 'Paleta_cocida': 'FLOAT',
 'Salchich¢n': 'FLOAT',
 'Salame': 'FLOAT',
 'Aceite_de_girasol': 'FLOAT',
 'Margarina_para_cocinar': 'FLOAT',
 'Leche_fluida': 'FLOAT',
 'Leche_en_polvo_entera': 'FLOAT',
 'Queso_crema': 'FLOAT',
 'Queso_cuartirolo': 'FLOAT',
 'Queso_de_rallar': 'FLOAT',
 'Manteca': 'FLOAT',
 'Yogur': 'FLOAT',
 'Dulce_de_leche': 'FLOAT',
 'Huevo': 'FLOAT',
 'Manzana': 'FLOAT',
 'Naranja': 'FLOAT',
 'Banana': 'FLOAT',
 'Pera': 'FLOAT',
 'Batata': 'FLOAT',
 'Papa': 'FLOAT',
 'Acelga': 'F

Inserto los precios de los productos en la base de datos

In [8]:
conn = psycopg2.connect(
    host="host.docker.internal",
    database="variacion",
    user="postgres",
    password="postgres" # Cambiar por la contraseña de la base de datos creada
)

In [9]:
# Conexión a la base de datos
conn = psycopg2.connect(
    host="host.docker.internal",
    database="variacion",
    user="postgres",
    password="postgres" # Cambiar por la contraseña de la base de datos creada
)
# Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute(
            f"CREATE TABLE IF NOT EXISTS precios ({', '.join([f'{columna} {tipo}' for columna, tipo in tabla.items()])})"
        )
    conn.commit()


# Insertar los datos en la tabla
with conn.cursor() as cur:
    columnas = []
    valores = []
    for columna, valor in listado.items():
        columnas.append(columna)
        valores.append(valor)
        
    data_fecha = cur.execute("SELECT fecha FROM precios WHERE fecha = %s", (valores[0],))
    data_fecha = cur.fetchone()
    print(data_fecha)
    if data_fecha == None:
        print('no hay fecha')
        print('Cargando datos') 
        query = f"INSERT INTO precios ({', '.join(columnas)}) VALUES ({', '.join(['%s'] * len(valores))})"
        cur.execute(query, valores)
        conn.commit()
    else:
        print('Ya se cargaron los datos de hoy')




# Cerrar la conexión a la base de datos
cur.close()
conn.close()

(datetime.date(2023, 3, 27),)
Ya se cargaron los datos de hoy


Traer los productos de la canasta basica desde la tabla de postgres para almacenarlos en CSV o EXCEL. Formatos alternativos para el archivo de salida.

In [10]:
# Conexión a la base de datos
conn = psycopg2.connect(
    host="host.docker.internal",
    database="variacion",
    user="postgres",
    password="postgres"
)

# Obtener los datos de la tabla
with conn.cursor() as cur:
    cur.execute("SELECT * FROM precios")
    rows = cur.fetchall()
    conn.commit()


if "nan" in rows:
    print("hay un Nan en la tabla")
    
# crear un dataframe si no existe o actualizarlo
try:
    df = pd.read_csv('./datos/precios.csv')
    
    for row in rows:
        print(f"Listado desde guardar_csv_excel \n {row}")
        fecha = row[1].strftime('%Y-%m-%d')
        print(fecha)  
        
        if fecha not in df['fecha'].values:
            df = df.append(pd.DataFrame([row], columns=[desc[0] for desc in cur.description]), ignore_index=True)
            df['fecha'] = pd.to_datetime(df['fecha'])
            df = df.sort_values(by='fecha')
           
            df = df.drop_duplicates()
            df = df.reset_index(drop=True)
            print('actualizando datos')
        else:
            print('no hay datos nuevos')
            continue
    
except FileNotFoundError:
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
    print('creando dataframe') 
   






#Cerrar la conexión a la base de datos
cur.close()
conn.close()

# Guardar el dataframe en un archivo csv
df.to_csv('./datos/precios.csv', index=False)

# guardar el dataframe en un archivo excel
df.to_excel('./datos/precios.xlsx', index=False)

df.tail(10)

Listado desde guardar_csv_excel 
 (8, datetime.date(2023, 3, 18), 1027.14, 198.49, 349.99, 201.75, 229.02, 223.89, 355.5, 1599.9, 1049.9, 429.9, 1499.9, 999.9, 1299.9, 299.9, 1049.9, 479.9, 1690.0, 1220.0, 1909.0, 1049.0, 4263.33, 574.74, 277.05, 242.15, 1485.55, 264.2, 1469.0, 2699.0, 1592.0, 236.25, 736.12, 1361.0, 499.0, 499.0, 419.0, 199.0, 479.0, 239.0, 329.0, 229.0, 199.0, 899.0, 699.0, 249.0, 199.0, 614.16, 509.04, 1313.15, 304.22, 726.0, 581.26, 316.3, 640.49, 179.6, 0.0, 92.52, 224.67, 117.5, 397.0, 454.0, 830.58, 770.85, 418.56)
2023-03-18
no hay datos nuevos
Listado desde guardar_csv_excel 
 (9, datetime.date(2023, 3, 19), 1027.14, 198.49, 349.99, 201.75, 229.02, 223.89, 355.5, 1599.9, 1049.9, 429.9, 1499.9, 999.9, 1299.9, 299.9, 1049.9, 479.9, 1690.0, 1220.0, 1909.0, 1049.0, 4263.33, 574.74, 277.05, 242.15, 1485.55, 264.2, 1469.0, 2699.0, 1592.0, 236.25, 736.12, 1361.0, 499.0, 499.0, 419.0, 199.0, 479.0, 239.0, 329.0, 229.0, 199.0, 899.0, 699.0, 249.0, 199.0, 614.16, 509.04

,id,fecha,pan_frances,galletitas_dulces,galletitas_de_agua,harina_de_trigo,harina_de_maiz,arroz,fideos_secos,asado,carnaza_comun,hueso_con_carne,paleta,carne_picada,nalga,higado,pechito_de_cerdo,pollo,carne_de_pescado,mortadela,paleta_cocida,salchich¢n,salame,aceite_de_girasol,margarina_para_cocinar,leche_fluida,leche_en_polvo_entera,queso_crema,queso_cuartirolo,queso_de_rallar,manteca,yogur,dulce_de_leche,huevo,manzana,naranja,banana,pera,batata,papa,acelga,cebolla,choclo,lechuga,tomate_perita,zanahoria,zapallo,tomate_envasado,arvejas_en_lata,lentejas_secas,azucar,dulce_de_batata,mermelada,sal_fina,mayonesa,vinagre,caldo_concentrado,gaseosas,jugos_concentrados,soda,cerveza,vino,cafe,yerba,te_en_saquitos
28,1,2023-03-18,1027.14,198.49,349.99,201.75,229.02,223.89,355.5,1599.9,1049.9,429.9,1499.9,999.9,1299.9,299.9,1049.9,479.9,1690.0,1220.0,1909.0,1049.0,4263.33,574.74,277.05,242.15,1485.55,264.2,1469.0,2699.0,1592.0,236.25,736.12,1361.0,499.0,499.0,419.0,199.0,479.0,239.0,329.0,229.0,199.0,899.0,699.0,249.0,199.0,614.16,509.04,1313.15,304.22,726.0,581.26,316.3,640.49,179.6,0.0,92.52,224.67,117.5,397.0,454.0,830.58,770.85,418.56
29,9,2023-03-19,1027.14,198.49,349.99,201.75,229.02,223.89,355.5,1599.9,1049.9,429.9,1499.9,999.9,1299.9,299.9,1049.9,479.9,1690.0,1220.0,1909.0,1049.0,4263.33,574.74,277.05,242.15,1485.55,264.2,1469.0,2699.0,1592.0,236.25,736.12,1361.0,499.0,499.0,419.0,199.0,479.0,239.0,329.0,229.0,199.0,899.0,699.0,249.0,199.0,614.16,509.04,1313.15,304.22,726.0,581.26,316.3,640.49,179.6,0.0,92.52,224.67,117.5,397.0,454.0,830.58,770.85,418.56
30,13,2023-03-20,1027.14,198.49,349.99,201.75,229.02,223.89,355.5,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,1049.9,479.9,1690.0,1630.0,1909.0,1325.0,4263.33,574.74,277.05,242.15,1485.55,264.2,1469.0,1999.0,1592.0,236.25,736.12,1361.0,499.0,499.0,449.0,249.0,479.0,269.0,329.0,229.0,199.0,899.0,699.0,249.0,169.0,614.16,509.04,1313.15,304.22,726.0,581.26,316.3,640.49,179.6,0.0,92.52,224.67,117.5,397.0,454.0,830.58,770.85,418.56
31,15,2023-03-21,1071.42,198.49,349.99,201.75,229.02,223.89,355.5,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,1049.9,479.9,1690.0,1630.0,1909.0,1049.0,4263.33,574.74,277.05,242.15,1485.55,264.2,1469.0,1999.0,1592.0,236.25,736.12,1361.0,499.0,499.0,449.0,249.0,479.0,269.0,319.0,229.0,199.0,999.0,699.0,249.0,169.0,614.16,509.04,1313.15,304.22,726.0,581.26,316.3,640.49,179.6,0.0,92.52,224.67,117.5,397.0,454.0,830.58,770.85,418.56
32,16,2023-03-22,1071.42,198.49,349.99,201.75,229.02,223.89,355.5,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,1049.9,479.9,1690.0,1630.0,1909.0,1049.0,4263.33,574.74,277.05,242.15,1485.55,264.2,1469.0,1999.0,1592.0,236.25,736.12,1361.0,499.0,499.0,449.0,249.0,499.0,269.0,319.0,229.0,199.0,999.0,699.0,249.0,169.0,614.16,509.04,1444.47,304.22,726.0,581.26,316.3,640.49,179.6,0.0,92.52,224.67,117.5,397.0,454.0,830.58,770.85,418.56
33,17,2023-03-23,1071.42,198.49,349.99,201.75,229.02,223.89,355.5,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,1049.9,479.9,1690.0,1630.0,1909.0,1049.0,4263.33,574.74,277.05,242.15,1485.55,264.2,1469.0,1999.0,1592.0,236.25,736.12,1361.0,499.0,599.0,449.0,249.0,499.0,269.0,329.0,199.0,249.0,1099.0,699.0,279.0,169.0,614.16,509.04,1444.47,304.22,726.0,581.26,316.3,640.49,179.6,0.0,92.52,224.67,117.5,397.0,454.0,830.58,770.85,418.56
34,18,2023-03-24,1071.42,198.49,349.99,201.75,229.02,223.89,355.5,949.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,829.9,479.9,1690.0,1630.0,1909.0,1049.0,4263.33,574.74,277.05,242.15,1485.55,264.2,1469.0,1999.0,1592.0,236.25,736.12,1361.0,499.0,599.0,449.0,249.0,499.0,269.0,329.0,199.0,199.0,1199.0,699.0,279.0,169.0,614.16,509.04,1444.47,304.22,726.0,581.26,316.3,640.49,179.6,0.0,92.52,224.67,117.5,397.0,454.0,830.58,770.85,418.56
35,19,2023-03-25,1071.42,198.49,349.99,201.75,229.02,223.89,355.5,949.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,829.9,479.9,1690.0,1630.0,1909.0,1049.0,4263.33,574.74,277.05,242.15,1485.55,264.2,1469.0,1999.0,1592.0,236.25,736.12,1361.0,499.0,599.0,449.0,249

Pequeño analisis para constrastar con los numeros que arrojan los datos en la tabla de Data Studio.

In [11]:
listado_precios = pd.read_csv('./datos/precios.csv', sep=',', encoding='latin-1')

lista_variaciones = {}
for producto in listado_precios:
    if producto == 'fecha':
        continue
    elif producto == 'id':
        continue
    else:
        variacion = (listado_precios[producto].iloc[-1] / listado_precios[producto].iloc[0] - 1) * 100
        lista_variaciones[producto] = variacion
        
precio_anterior = sum(listado_precios.iloc[0, 2:])
precio_actual = sum(listado_precios.iloc[-1, 2:])
print(f"El precio anterior es ${precio_anterior}")
print(f"El precio actual es ${precio_actual}")
suma = sum(lista_variaciones.values()) / len(lista_variaciones)
print('La variación media de precio de los productos de la canasta básica es del', round(suma, 2), '%')


El precio anterior es $43354.810000000005
El precio actual es $48171.770000000004
La variación media de precio de los productos de la canasta básica es del 12.77 %


### Creamos un csv en formato de lista larga para poder usarlo correctamente en data studio.

El formato de lista larga nos permite tener un registro por producto y por fecha y facilitar su visibilidad en data studio.

In [12]:
lista_tabla = df.columns.to_list()

lista_tabla.remove('id')
lista_tabla.remove('fecha')
lista_tabla




['pan_frances',
 'galletitas_dulces',
 'galletitas_de_agua',
 'harina_de_trigo',
 'harina_de_maiz',
 'arroz',
 'fideos_secos',
 'asado',
 'carnaza_comun',
 'hueso_con_carne',
 'paleta',
 'carne_picada',
 'nalga',
 'higado',
 'pechito_de_cerdo',
 'pollo',
 'carne_de_pescado',
 'mortadela',
 'paleta_cocida',
 'salchich¢n',
 'salame',
 'aceite_de_girasol',
 'margarina_para_cocinar',
 'leche_fluida',
 'leche_en_polvo_entera',
 'queso_crema',
 'queso_cuartirolo',
 'queso_de_rallar',
 'manteca',
 'yogur',
 'dulce_de_leche',
 'huevo',
 'manzana',
 'naranja',
 'banana',
 'pera',
 'batata',
 'papa',
 'acelga',
 'cebolla',
 'choclo',
 'lechuga',
 'tomate_perita',
 'zanahoria',
 'zapallo',
 'tomate_envasado',
 'arvejas_en_lata',
 'lentejas_secas',
 'azucar',
 'dulce_de_batata',
 'mermelada',
 'sal_fina',
 'mayonesa',
 'vinagre',
 'caldo_concentrado',
 'gaseosas',
 'jugos_concentrados',
 'soda',
 'cerveza',
 'vino',
 'cafe',
 'yerba',
 'te_en_saquitos']

In [13]:
lista_larga = df.melt(id_vars=['fecha'], value_vars=lista_tabla, var_name='producto', value_name='precio')
lista_larga[-2:]['producto']

2392    te_en_saquitos
2393    te_en_saquitos
Name: producto, dtype: object

In [14]:
lista_larga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2394 entries, 0 to 2393
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fecha     2394 non-null   object 
 1   producto  2394 non-null   object 
 2   precio    2394 non-null   float64
dtypes: float64(1), object(2)
memory usage: 56.2+ KB


In [15]:
lista_larga.to_csv('./datos/precios_lista_larga.csv', index=False)
lista_larga['fecha'] = pd.to_datetime(lista_larga['fecha'])

### Subimos la lista_larga a una DDBB en postgres online. 
 

la base esta en azure

In [16]:
from certificados_ddbb import ddbb_pass

In [17]:
fecha = dt.datetime.now().strftime("%Y-%m-%d")
fecha

'2023-03-27'

In [18]:
print(type(lista_larga.loc[1, "fecha"].strftime("%Y-%m-%d")))

<class 'str'>


In [25]:


# Conexión a la base de datos
conn = psycopg2.connect(
    host="canasta-2.crdqtsbdpist.us-east-2.rds.amazonaws.com",
    database="postgres",
    user="postgres",
    password=ddbb_pass
)

# Crear la tabla si no existe
with conn.cursor() as cur:
    
    cur.execute(f"CREATE TABLE IF NOT EXISTS precios_lista_larga (fecha DATE, producto VARCHAR(255), precio FLOAT, id SERIAL PRIMARY KEY)")
    conn.commit()

# Insertar los datos en la tabla



    for i in lista_larga.index:
            


            if lista_larga.loc[i, 'fecha'].strftime("%Y-%m-%d") == dt.datetime.now().strftime("%Y-%m-%d"):
                
                print('es hoy')
                producto_fecha = cur.execute("SELECT fecha, producto FROM precios_lista_larga WHERE fecha = %s AND producto = %s", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto']))
                producto_fecha = cur.fetchone()
                print(producto_fecha)
            
                if producto_fecha == None:
                    print(f'no existe, cargando {lista_larga.loc[i, "producto"]}')
                        
                    cur.execute("INSERT INTO precios_lista_larga (fecha, producto, precio) VALUES (%s, %s, %s)", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto'], lista_larga.loc[i, 'precio']))
                    conn.commit()
                else:
                    print(f'ya existe {lista_larga.loc[i, "producto"]}, {lista_larga.loc[i, "fecha"]}')
                    continue
            else:
                
                continue



#Cerrar la conexión a la base de datos
cur.close()
conn.close()

print('Datos cargados')
    

es hoy
None
no existe, cargando pan_frances
es hoy
None
no existe, cargando galletitas_dulces
es hoy
None
no existe, cargando galletitas_de_agua
es hoy
None
no existe, cargando harina_de_trigo
es hoy
None
no existe, cargando harina_de_maiz
es hoy
None
no existe, cargando arroz
es hoy
None
no existe, cargando fideos_secos
es hoy
None
no existe, cargando asado
es hoy
None
no existe, cargando carnaza_comun
es hoy
None
no existe, cargando hueso_con_carne
es hoy
None
no existe, cargando paleta
es hoy
None
no existe, cargando carne_picada
es hoy
None
no existe, cargando nalga
es hoy
None
no existe, cargando higado
es hoy
None
no existe, cargando pechito_de_cerdo
es hoy
None
no existe, cargando pollo
es hoy
None
no existe, cargando carne_de_pescado
es hoy
None
no existe, cargando mortadela
es hoy
None
no existe, cargando paleta_cocida
es hoy
None
no existe, cargando salchich¢n
es hoy
None
no existe, cargando salame
es hoy
None
no existe, cargando aceite_de_girasol
es hoy
None
no existe, carga

In [5]:
from datetime import datetime, timedelta, timezone

hora_actual = datetime.now(timezone.utc).hour
print(hora_actual)

def horario_branch_check():
    if hora_actual < 15 + 3 and hora_actual >= 8 + 3:
        print('es horario')
        
    else:
        print('no es horario')
        
horario_branch_check()

11
es horario
